In [ ]:
import pandas as pd
seed_value = 2043
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}

    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping


        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)

        return X_encoded

In [ ]:
#syn_df=pd.read_csv('C:/Users/Joy/Python-CODE/GAN-Privacy/Colab-Data/Colab-data-5000/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset5000.csv')
syn_df=pd.read_csv('drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset5000.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,-0.439699,0.342073,-0.439699,-0.439699,0.000000,-0.439699,0.000000,2.499218,-0.439699,-0.439699,-0.439699,0.342073,0.342073,0.342073,0.000000,0.000000,0.867426,-0.439699,0.342073,-0.439699
1,-0.439699,0.867426,0.342073,-0.439699,0.342073,0.867426,-0.439699,0.867426,-0.439699,-0.439699,0.000000,-0.439699,0.867426,0.342073,-0.439699,-0.439699,0.000000,0.000000,0.000000,-0.439699
2,-0.439699,0.342073,-0.439699,-0.439699,0.342073,0.342073,-0.439699,-0.439699,-0.439699,-0.439699,0.000000,-0.439699,-0.439699,0.000000,0.000000,0.342073,0.000000,0.000000,0.000000,-0.439699
3,-0.439699,0.867426,-0.439699,0.342073,0.000000,-0.439699,-0.439699,-0.439699,-0.439699,0.000000,-0.439699,-0.439699,0.342073,0.342073,0.342073,-0.439699,0.342073,0.342073,0.342073,0.342073
4,-0.439699,0.000000,-0.439699,-0.439699,0.342073,0.342073,0.342073,-0.439699,0.342073,0.000000,0.000000,0.000000,-0.439699,0.000000,0.000000,0.867426,-0.439699,0.000000,-0.439699,1.298190


In [ ]:
#org_test_df=pd.read_csv('C:/Users/Joy/Python-CODE/GAN-Privacy/Colab-Data/Colab-data-5000/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataseat5000.csv')
org_test_df=pd.read_csv('drive/My Drive/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataseat5000.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,-0.358999,0.039822,0.031013,0.200916,1.810602,-0.605655,1.410377,0.973277,0.272236,0.973277,-0.358999,-0.928147,0.000000,0.441866,0.242130,0.441866,-0.276281,-0.203610,-0.308443,0.201986
1,1.410377,-0.111723,-0.018455,0.200916,-0.462942,0.221536,0.000000,0.000000,-0.643134,0.000000,0.000000,-0.864143,-0.557091,0.326591,0.242130,0.326591,-0.249514,-0.077157,-0.308443,0.201986
2,-0.358999,-0.358999,0.004923,0.000000,-0.358999,-0.647981,0.000000,-0.358999,0.409120,-0.358999,-0.358999,0.172797,0.622058,0.289240,-0.284555,0.289240,-0.249514,0.480250,-0.308443,-0.872765
3,0.000000,0.900476,-0.018455,0.200916,-0.242985,0.517807,0.323957,-0.242985,0.683962,0.000000,0.000000,0.441866,0.323957,0.000000,0.242130,0.000000,-0.249514,-0.214692,-0.308443,0.201986
4,-0.358999,0.323957,-0.018455,0.200916,-0.358999,-0.013731,0.000000,-0.242985,0.000000,-0.557091,0.000000,-0.561457,-0.358999,0.000000,-0.284555,0.000000,-0.249514,-0.688589,-0.308443,0.201986


In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10,
                          n_repeats=3,
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1270 23 74 133
pod:  0.642512077294686
pof:  0.017788089713843776
AUC:  0.8123619937904211
recall:  0.642512077294686
precision:  0.8525641025641025
F1-Score:  0.7327823691460055
accuracy:  0.9353333333333333


In [ ]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



1290 3 111 96
pod:  0.463768115942029
pof:  0.002320185614849188
AUC:  0.73072396516359
recall:  0.463768115942029
precision:  0.9696969696969697
F1-Score:  0.6274509803921569
accuracy:  0.924


In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1292 1 199 8
pod:  0.03864734299516908
pof:  0.0007733952049497294
AUC:  0.5189369738951096
recall:  0.03864734299516908
precision:  0.8888888888888888
F1-Score:  0.07407407407407408
accuracy:  0.8666666666666667


In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1292 1 202 5
pod:  0.024154589371980676
pof:  0.0007733952049497294
AUC:  0.5116905970835155
recall:  0.024154589371980676
precision:  0.8333333333333334
F1-Score:  0.046948356807511735
accuracy:  0.8646666666666667


In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1289 4 172 35
pod:  0.16908212560386474
pof:  0.0030935808197989174
AUC:  0.5829942723920329
recall:  0.16908212560386474
precision:  0.8974358974358975
F1-Score:  0.2845528455284553
accuracy:  0.8826666666666667


In [ ]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1292 1 177 30
pod:  0.14492753623188406
pof:  0.0007733952049497294
AUC:  0.5720770705134671
recall:  0.14492753623188406
precision:  0.967741935483871
F1-Score:  0.25210084033613445
accuracy:  0.8813333333333333


In [ ]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0)

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


1293 0 207 0
pod:  0.0
pof:  0.0
AUC:  0.5
recall:  0.0
precision:  nan
F1-Score:  nan
accuracy:  0.862


<ipython-input-13-67a39a747352>:24: RuntimeWarning: invalid value encountered in long_scalars
  precision=tp/(tp+fp)


In [ ]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Epoch 1/30
88/88 [==============================] - 4s 7ms/step - loss: 0.3084 - accuracy: 0.8857 - val_loss: 0.1853 - val_accuracy: 0.9357
Epoch 2/30
88/88 [==============================] - 0s 5ms/step - loss: 0.1999 - accuracy: 0.9243 - val_loss: 0.1759 - val_accuracy: 0.9414
Epoch 3/30
88/88 [==============================] - 0s 4ms/step - loss: 0.1734 - accuracy: 0.9339 - val_loss: 0.1463 - val_accuracy: 0.9514
Epoch 4/30
88/88 [==============================] - 0s 5ms/step - loss: 0.1574 - accuracy: 0.9396 - val_loss: 0.1574 - val_accuracy: 0.9500
Epoch 5/30
88/88 [==============================] - 0s 4ms/step - loss: 0.1492 - accuracy: 0.9429 - val_loss: 0.1431 - val_accuracy: 0.9500
Epoch 6/30
88/88 [==============================] - 0s 4ms/step - loss: 0.1447 - accuracy: 0.9436 - val_loss: 0.1443 - val_accuracy: 0.9543
Epoch 7/30
88/88 [==============================] - 0s 5ms/step - loss: 0.1378 - accuracy: 0.9436 - val_loss: 0.1619 - val_accuracy: 0.9471
Epoch 8/30
47/47 [==

In [ ]:
#RNN
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers.legacy import Adam

import numpy as np
max_features = 10000 # number of words to consider as features
#create model
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
#model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
history = model.fit(X_encoded, y_syn_df, epochs=5, batch_size=128, validation_split=0.2)
y2_pred = model.predict(X_encoded_test)
y22_pred=y2_pred.round()


from sklearn.metrics import confusion_matrix
tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          320000    
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 328353 (1.25 MB)
Trainable params: 328353 (1.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/5
22/22 [==============================] - 3s 35ms/step - loss: 0.5752 - accuracy: 0.8446 - val_loss: 0.3737 - val_accuracy: 0.8743
Epoch 2/5
22/22 [==============================] - 0s 8ms/step - loss: 0.4180 - accuracy: 0.8446 - val_loss: 0.3720 - val_accuracy: 0.874